In [1]:
#Within-task pre-training (ITPT) or mlm pretraining demo using deberta-v3-large huggingface LM 

In [2]:
!nvidia-smi

/bin/bash: /home/apsisdev/anaconda3/envs/eblict/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Sun Aug  7 21:07:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.05    Driver Version: 510.73.05    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
|  0%   55C    P0    53W / 370W |    364MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+-----------

In [3]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

from transformers import (AutoModel,AutoModelForMaskedLM, DebertaV2ForMaskedLM,
                          AutoTokenizer, LineByLineTextDataset,DebertaV2TokenizerFast,
                          DataCollatorForLanguageModeling,
                          Trainer, TrainingArguments)

In [4]:
model_name = 'microsoft/deberta-v3-large'
# trained_tokenizer = '/home/apsisdev/mobassir/feedback-prize-effectiveness/mlm/debertav3new-tokenizer'
model = AutoModelForMaskedLM.from_pretrained(model_name)
for param in model.base_model.parameters():
    param.requires_grad = False
    
tokenizer = DebertaV2TokenizerFast.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)
print(tokenizer.is_fast)
tokenizer.save_pretrained('/home/apsisdev/mobassir/feedback-prize-effectiveness/mlm/itpt_deberta_v3large_tokenizer');

Some weights of the model checkpoint at microsoft/deberta-v3-large were not used when initializing DebertaV2ForMaskedLM: ['deberta.embeddings.position_embeddings.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.weight', 'mask_predictions.dense.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.classifier.bias']
- This IS expected if you are initializing DebertaV2ForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification mode

True


In [5]:
%%time

train_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="/home/apsisdev/mobassir/feedback-prize-effectiveness/mlm/train.txt", #train text file here
    block_size=512)

valid_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="/home/apsisdev/mobassir/feedback-prize-effectiveness/mlm/test.txt", #valid text file here
    block_size=512)


data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

CPU times: user 4min 20s, sys: 2.75 s, total: 4min 23s
Wall time: 24.3 s


In [6]:

##Num examples = 115413

training_args = TrainingArguments(
    output_dir="./deberta_v3large_chk", #select model path for checkpoint
    overwrite_output_dir=True,
    num_train_epochs=15*2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy= 'steps',
   
    save_total_limit=2,
    save_steps = 108210*2,
    eval_steps= 54105,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    load_best_model_at_end =True,
    prediction_loss_only=True,
    report_to = "none")


# training_args = TrainingArguments(
#         output_dir="./deberta_v3_large_chk",
#         evaluation_strategy="epoch",
#         learning_rate=2e-5,
#         weight_decay=0.01,
#         save_strategy='no',
#         per_device_train_batch_size=20,
#         num_train_epochs=15,
#         # report_to="wandb",
#         run_name=f'output-mlm-exp1',
#         # logging_dir='./logs',
#         lr_scheduler_type='cosine',
#         warmup_ratio=0.2,
#         fp16=True,
#         logging_steps=500,
#         gradient_accumulation_steps=1
#     )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset)

In [7]:
%%time

trainer.train()
trainer.save_model('/home/apsisdev/mobassir/feedback-prize-effectiveness/mlm/itpt_debertav3large')

***** Running training *****
  Num examples = 115413
  Num Epochs = 30
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 216420


Step,Training Loss,Validation Loss
54105,2.756500,2.415445
108210,2.668500,2.324528
162315,2.642600,2.290476
216420,2.625000,2.275367


***** Running Evaluation *****
  Num examples = 28853
  Batch size = 16
***** Running Evaluation *****
  Num examples = 28853
  Batch size = 16
***** Running Evaluation *****
  Num examples = 28853
  Batch size = 16
***** Running Evaluation *****
  Num examples = 28853
  Batch size = 16
Saving model checkpoint to ./deberta_v3large_chk/checkpoint-216420
Configuration saved in ./deberta_v3large_chk/checkpoint-216420/config.json
Model weights saved in ./deberta_v3large_chk/checkpoint-216420/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./deberta_v3large_chk/checkpoint-216420 (score: 2.275367259979248).
Saving model checkpoint to /home/apsisdev/mobassir/feedback-prize-effectiveness/mlm/itpt_debertav3large
Configuration saved in /home/apsisdev/mobassir/feedback-prize-effectiveness/mlm/itpt_debertav3large/config.json
Model weights saved in /home/apsisdev/mobassir/feedback-prize-effectiveness/mlm/itpt_debertav3l

CPU times: user 2d 3h 54min 10s, sys: 1min 21s, total: 2d 3h 55min 32s
Wall time: 1d 22h 27min 37s
